In [1]:
from sqlalchemy import select
from sqlalchemy.orm import Session

import networkx as nx
import pandas as pd
import numpy as np
import json
import time


In [2]:
from art_graph import directories
from art_graph.cinema_data_providers.imdb_non_commercial.locations import get_sqlite_engine

from art_graph.cinema_data_providers.imdb_non_commercial import (
    imdb_non_commercial_pydantic_models as imdb_pyd,
    imdb_non_commercial_orm_models as imdb_orm,
)

from art_graph.cinema_data_providers.imdb_non_commercial.queries import (
    compute_graph,
    full_graph_query,
    curated_graph_query,
    famous_query,
)

from art_graph.cinegraph import random_path
from art_graph.cinegraph.node_types import PersonNode, WorkNode
from art_graph.cinema_data_providers.imdb_non_commercial import (
    imdb_non_commercial_orm_models as imdb_orm,
)
from art_graph.cinema_data_providers.imdb_non_commercial.name_lookup import node_lookup

In [3]:
actor_df = pd.read_csv(directories.data("actors.csv"))
primary_names = list(actor_df.name)

movie_df = pd.read_csv(directories.data("movies.csv"))
original_titles = list(movie_df.title)

In [4]:
engine = get_sqlite_engine("IM02.db")

In [5]:
# Start a session
session = Session(bind=engine)


In [6]:
t0 = time.time()
query = curated_graph_query(session, primary_names, original_titles)
g_currated = compute_graph(query)
print(f"Total time {(time.time() - t0) / 60 : .2f}")

Total time  0.34


In [7]:
len(g_currated)

264

In [8]:
query = session.query(imdb_orm.NameBasics).filter(imdb_orm.NameBasics.nconst == 1)

In [9]:
query.all()[0].primaryName

'Fred Astaire'

In [10]:
path_maker = random_path.PathMaker(g_currated)

In [11]:
r = np.random.RandomState(42)

In [12]:
for _ in range(6):
    a, b = path_maker.make_game(1, r)
    path = nx.shortest_path(g_currated, a, b)
    print(" -> ".join([node_lookup(session, node) for node in path]))

Kathy Bates -> Titanic -> Leonardo DiCaprio
Michael Caine -> The Dark Knight -> Morgan Freeman
Michael Fassbender -> 12 Years a Slave -> Brad Pitt
Kathy Bates -> Titanic -> Leonardo DiCaprio
Renée Zellweger -> Chicago -> Richard Gere
Alec Baldwin -> The Aviator -> Leonardo DiCaprio


In [13]:
for _ in range(6):
    a, b = path_maker.make_game(2, r)
    path = nx.shortest_path(g_currated, a, b)
    print(" -> ".join([node_lookup(session, node) for node in path]))

John Malkovich -> Places in the Heart -> Ed Harris -> A Beautiful Mind -> Christopher Plummer
Amy Adams -> Her -> Joaquin Phoenix -> Gladiator -> Russell Crowe
Christian Bale -> The Dark Knight -> Morgan Freeman -> Million Dollar Baby -> Hilary Swank
Forest Whitaker -> Platoon -> Willem Dafoe -> The English Patient -> Ralph Fiennes
Bill Murray -> The Grand Budapest Hotel -> Edward Norton -> Fight Club -> Brad Pitt
Holly Hunter -> The Incredibles -> Samuel L. Jackson -> Django Unchained -> Christoph Waltz


In [14]:
for _ in range(6):
    a, b = path_maker.make_game(3, r)
    path = nx.shortest_path(g_currated, a, b)
    print(" -> ".join([node_lookup(session, node) for node in path]))

Colin Firth -> The King's Speech -> Helena Bonham Carter -> Howards End -> Emma Thompson -> Sense and Sensibility -> Kate Winslet
Ralph Fiennes -> Schindler's List -> Liam Neeson -> Gangs of New York -> Leonardo DiCaprio -> The Aviator -> Cate Blanchett
Gene Hackman -> Unforgiven -> Morgan Freeman -> The Dark Knight -> Gary Oldman -> JFK -> Kevin Costner
Russell Crowe -> A Beautiful Mind -> Ed Harris -> Apollo 13 -> Kevin Bacon -> Mystic River -> Laura Linney
Johnny Depp -> Chocolat -> Juliette Binoche -> The English Patient -> Ralph Fiennes -> Schindler's List -> Ben Kingsley
Joaquin Phoenix -> Gladiator -> Russell Crowe -> A Beautiful Mind -> Ed Harris -> Places in the Heart -> John Malkovich


In [15]:
for _ in range(6):
    a, b = path_maker.make_game(4, r)
    path = nx.shortest_path(g_currated, a, b)
    print(" -> ".join([node_lookup(session, node) for node in path]))

Scarlett Johansson -> Lost in Translation -> Bill Murray -> Tootsie -> Dustin Hoffman -> Rain Man -> Tom Cruise -> A Few Good Men -> Jack Nicholson
Nicole Kidman -> The Hours -> Julianne Moore -> Magnolia -> Philip Seymour Hoffman -> Scent of a Woman -> Al Pacino -> The Insider -> Christopher Plummer
Ralph Fiennes -> The Grand Budapest Hotel -> Bill Murray -> Tootsie -> Dustin Hoffman -> Rain Man -> Tom Cruise -> Magnolia -> Philip Seymour Hoffman
Brad Pitt -> The Tree of Life -> Sean Penn -> Mystic River -> Tim Robbins -> The Shawshank Redemption -> Morgan Freeman -> The Dark Knight -> Gary Oldman
Liam Neeson -> Schindler's List -> Ralph Fiennes -> The Grand Budapest Hotel -> Bill Murray -> Lost in Translation -> Scarlett Johansson -> Her -> Joaquin Phoenix
Christian Bale -> The Dark Knight -> Morgan Freeman -> The Shawshank Redemption -> Tim Robbins -> Mystic River -> Kevin Bacon -> Apollo 13 -> Gary Sinise


In [16]:
for _ in range(6):
    a, b = path_maker.make_game(5, r)
    path = nx.shortest_path(g_currated, a, b)
    print(" -> ".join([node_lookup(session, node) for node in path]))

Gary Oldman -> The Dark Knight -> Morgan Freeman -> The Shawshank Redemption -> Tim Robbins -> Mystic River -> Sean Penn -> The Tree of Life -> Brad Pitt -> Fight Club -> Edward Norton
Judi Dench -> Chocolat -> Johnny Depp -> Finding Neverland -> Dustin Hoffman -> Rain Man -> Tom Cruise -> A Few Good Men -> Kevin Bacon -> Apollo 13 -> Ed Harris
Richard Gere -> Chicago -> John C. Reilly -> The Aviator -> Cate Blanchett -> Babel -> Brad Pitt -> The Tree of Life -> Jessica Chastain -> Interstellar -> Ellen Burstyn
Ian McKellen -> The Lord of the Rings: The Two Towers -> Cate Blanchett -> The Aviator -> Alec Baldwin -> Working Girl -> Kevin Spacey -> L.A. Confidential -> Russell Crowe -> Gladiator -> Joaquin Phoenix
Clint Eastwood -> Unforgiven -> Morgan Freeman -> The Shawshank Redemption -> Tim Robbins -> Mystic River -> Kevin Bacon -> A Few Good Men -> Jack Nicholson -> The Departed -> Leonardo DiCaprio
Kate Winslet -> Titanic -> Leonardo DiCaprio -> The Departed -> Matt Damon -> Saving

In [17]:
for _ in range(6):
    a, b = path_maker.make_game(6, r)
    path = nx.shortest_path(g_currated, a, b)
    print(" -> ".join([node_lookup(session, node) for node in path]))

Dustin Hoffman -> Rain Man -> Tom Cruise -> A Few Good Men -> Kevin Bacon -> Mystic River -> Tim Robbins -> The Shawshank Redemption -> Morgan Freeman -> The Dark Knight -> Gary Oldman -> JFK -> Kevin Costner
Octavia Spencer -> Being John Malkovich -> John Malkovich -> Places in the Heart -> Ed Harris -> Apollo 13 -> Kevin Bacon -> Mystic River -> Tim Robbins -> The Shawshank Redemption -> Morgan Freeman -> The Dark Knight -> Michael Caine
Nicole Kidman -> The Hours -> Julianne Moore -> Magnolia -> Tom Cruise -> A Few Good Men -> Kevin Bacon -> Mystic River -> Tim Robbins -> The Shawshank Redemption -> Morgan Freeman -> Unforgiven -> Clint Eastwood
Chiwetel Ejiofor -> 12 Years a Slave -> Brad Pitt -> Babel -> Cate Blanchett -> The Aviator -> Alec Baldwin -> Working Girl -> Harrison Ford -> The Fugitive -> Julianne Moore -> The Hours -> Meryl Streep
Chiwetel Ejiofor -> 12 Years a Slave -> Brad Pitt -> The Tree of Life -> Sean Penn -> Mystic River -> Tim Robbins -> The Shawshank Redempti

In [18]:
for _ in range(6):
    a, b = path_maker.make_game(7, r)
    path = nx.shortest_path(g_currated, a, b)
    print(" -> ".join([node_lookup(session, node) for node in path]))

Geoffrey Rush -> The King's Speech -> Colin Firth -> The English Patient -> Ralph Fiennes -> The Grand Budapest Hotel -> Bill Murray -> Tootsie -> Dustin Hoffman -> Rain Man -> Tom Cruise -> Magnolia -> Julianne Moore -> The Hours -> Nicole Kidman
Russell Crowe -> L.A. Confidential -> Kevin Spacey -> Working Girl -> Alec Baldwin -> The Aviator -> Leonardo DiCaprio -> Gangs of New York -> Daniel Day-Lewis -> In the Name of the Father -> Emma Thompson -> Howards End -> Anthony Hopkins -> The Silence of the Lambs -> Jodie Foster
Meryl Streep -> The Hours -> Julianne Moore -> Magnolia -> Tom Cruise -> A Few Good Men -> Kevin Bacon -> Mystic River -> Tim Robbins -> The Shawshank Redemption -> Morgan Freeman -> The Dark Knight -> Gary Oldman -> JFK -> Kevin Costner
Kevin Costner -> JFK -> Gary Oldman -> The Dark Knight -> Morgan Freeman -> The Shawshank Redemption -> Tim Robbins -> Mystic River -> Kevin Bacon -> A Few Good Men -> Tom Cruise -> Magnolia -> Philip Seymour Hoffman -> Scent of a